In [ ]:
#| default_exp core

# dialoghelper

In [ ]:
#| export
import json, importlib, linecache
from typing import Dict
from tempfile import TemporaryDirectory
from ipykernel_helper import *
from dataclasses import dataclass

from fastcore.utils import *
from fastcore.meta import delegates
from ghapi.all import *
from fastlite import *
from fastcore.xtras import asdict
from inspect import currentframe,Parameter,signature

In [ ]:
from IPython.display import display,Markdown

Re-export asdict:

In [ ]:
#| export
_all_ = ["asdict"]

In [ ]:
#| export
def get_db(ns:dict=None):
    app_path = Path('/app') if Path('/.dockerenv').exists() else Path('.')
    if os.environ.get('IN_SOLVEIT', False): dataparent,nm = app_path, 'data.db'
    else: dataparent,nm = Path('..'),'dev_data.db'
    db = database(dataparent/'data'/nm)
    dcs = [o for o in all_dcs(db) if o.__name__[0]!='_']
    if ns:
        for o in dcs: ns[o.__name__]=o
    return db

In [ ]:
# db.conn.filename

In [ ]:
db = get_db(globals())
dlg = db.t.dialog.selectone()
dlg

In [ ]:
#| export
def find_var(var:str):
    "Search for var in all frames of the call stack"
    frame = currentframe()
    while frame:
        dv = frame.f_globals.get(var, frame.f_locals.get(var, None))
        if dv: return dv
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
a = 1
find_var('a')

In [ ]:
#| export
def find_dialog_id():
    "Get the dialog id by searching the call stack for __dialog_id."
    return find_var('__dialog_id')

In [ ]:
__dialog_id = dlg.id

In [ ]:
find_dialog_id()

In [ ]:
#| export
def find_msgs(
    pattern:str='', # Optional text to search for
    msg_type:str=None, # optional limit by message type ('code', 'note', or 'prompt')
    limit:int=None, # Optionally limit number of returned items
    include_output:bool=True # Include output in returned dict?
):
    "Find messages in current specific dialog that contain the given information. To refer to a message found later, use its `sid` field (which is the pk)."
    did = find_dialog_id()
    db = get_db()
    res = db.t.message('did=? AND content LIKE ? ORDER BY mid', [did, f'%{pattern}%'], limit=limit)
    res = [asdict(o) for o in res if not msg_type or (msg_type==o.msg_type)]
    if not include_output:
        for o in res: o.pop('output', None)
    return res

In [ ]:
found = find_msgs('hello')
found[0]['content']

In [ ]:
find_msgs(msg_type='prompt', include_output=False)

In [ ]:
#| export
def find_msg_id():
    "Get the message id by searching the call stack for __dialog_id."
    return find_var('__msg_id')

In [ ]:
__msg_id = found[0]['sid']

In [ ]:
find_msg_id()

In [ ]:
#| export
def read_msg_ids():
    "Get all ids in current dialog."
    did = find_dialog_id()
    db = get_db()
    return [o.sid for o in db.t.message('did=?', [did], select='sid', order_by='mid')]

In [ ]:
#| export
def msg_idx():
    "Get relative index of current message in dialog."
    ids = read_msg_ids()
    return ids,ids.index(find_msg_id())

In [ ]:
ids,idx = msg_idx()
idx

In [ ]:
#| export
def read_msg(n:int=-1,     # Message index (if relative, +ve is downwards)
             relative:bool=True  # Is `n` relative to current message (True) or absolute (False)?
    ):
    "Get the message indexed in the current dialog."
    ids,idx = msg_idx()
    if relative:
        idx = idx+n
        if not 0<=idx<len(ids): return None
    else: idx = n
    db = get_db()
    return db.t.message.fetchone('sid=?', [ids[idx]])

In [ ]:
# Previous message relative to current
read_msg(-1)

In [ ]:
# Last message in dialog
read_msg(-1, relative=False)

In [ ]:
#| export
def _msg(
    time_run: str | None = '', # When was message executed
    is_exported: int | None = 0, # Export message to a module?
    skipped: int | None = 0, # Hide message from prompt?
    i_collapsed: int | None = 0, # Collapse input?
    o_collapsed: int | None = 0, # Collapse output?
    header_collapsed: int | None = 0, # Collapse heading section?
    pinned: int | None = 0 # Pin to context?
): ...

Placements = str_enum('Placements', 'add_after', 'add_before', 'update', 'at_start', 'at_end')

In [ ]:
#| export
@delegates(_msg)
def add_msg(
    content:str, # Content of the message (i.e the message prompt, code, or note text)
    msg_type: str='note', # Message type, can be 'code', 'note', or 'prompt'
    output:str='', # For prompts/code, initial output
    placement:str='add_after', # Can be 'add_after', 'add_before', 'update', 'at_start', 'at_end'
    sid:str=None, # sid (stable id -- pk) of message that placement is relative to (if None, uses current message)
    **kwargs
):
    "Add/update a message to the queue to show after code execution completes. Be sure to pass a `sid` (stable id) not a `mid` (which is used only for sorting, and can change)."
    assert msg_type in ('note', 'code', 'prompt'), "msg_type must be 'code', 'note', or 'prompt'."
    assert msg_type not in ('note') or not output, "'note' messages cannot have an output."
    run_cmd('add_msg', content=content, msg_type=msg_type, output=output, placement=placement, sid=sid, **kwargs)

In [ ]:
#| export
def _umsg(
    content:str|None = None, # Content of the message (i.e the message prompt, code, or note text)
    msg_type: str|None = None, # Message type, can be 'code', 'note', or 'prompt'
    output:str|None = None, # For prompts/code, the output
    time_run: str | None = None, # When was message executed
    is_exported: int | None = None, # Export message to a module?
    skipped: int | None = None, # Hide message from prompt?
    i_collapsed: int | None = None, # Collapse input?
    o_collapsed: int | None = None, # Collapse output?
    header_collapsed: int | None = None, # Collapse heading section?
    pinned: int | None = None # Pin to context?
): ...

In [ ]:
#| export
@delegates(_umsg)
def update_msg(
    msg:Optional[Dict]=None, # Dictionary of field keys/values to update
    sid:str=None, # sid (stable id -- pk) of message to update (if None, uses current message)
    **kwargs):
    """Update an existing message. Provide either `msg` OR field key/values to update.
    Use `content` param to update contents. Be sure to pass a `sid` (stable id -- the pk) not a `mid`
    (which is used only for sorting, and can change).
    Only include parameters to update--missing ones will be left unchanged."""
    kw = (msg or {}) | kwargs
    sid = kw.pop('sid', sid)
    if not sid: raise TypeError("update_msg needs either a dict message or `sid=...`")
    kw.pop('did', None)
    run_cmd('add_msg', placement='update', sid=sid, **kw)

In [ ]:
#| export
def add_html(
    html:str, # HTML to add to the DOM
):
    "Dynamically add HTML to the current web page. Supports HTMX attrs too."
    run_cmd('add_ft', html=html)

In [ ]:
#| export
def load_gist(gist_id:str):
    "Retrieve a gist"
    api = GhApi()
    if '/' in gist_id: *_,user,gist_id = gist_id.split('/')
    else: user = None
    return api.gists.get(gist_id, user=user)

In [ ]:
gistid = 'jph00/e7cfd4ded593e8ef6217e78a0131960c'
gist = load_gist(gistid)
gist.html_url

'https://gist.github.com/jph00/e7cfd4ded593e8ef6217e78a0131960c'

In [ ]:
#| export
def gist_file(gist_id:str):
    "Get the first file from a gist"
    gist = load_gist(gist_id)
    return first(gist.files.values())

In [ ]:
gfile = gist_file(gistid)
print(gfile.content[:100]+"…")

"This is a test module which makes some simple tools available."
__all__ = ["hi","whoami"]

testfoo=…


In [ ]:
#| export
def import_string(
    code:str, # Code to import as a module
    name:str  # Name of module to create
):
    with TemporaryDirectory() as tmpdir:
        path = Path(tmpdir) / f"{name}.py"
        path.write_text(code)
        # linecache.cache storage allows inspect.getsource() after tmpdir lifetime ends
        linecache.cache[str(path)] = (len(code), None, code.splitlines(keepends=True), str(path))
        spec = importlib.util.spec_from_file_location(name, path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[name] = module
        spec.loader.exec_module(module)
        return module

In [ ]:
#| export
empty = Parameter.empty

def is_usable_tool(func:callable):
    "True if the function has a docstring and all parameters have types, meaning that it can be used as an LLM tool."    
    if not func.__doc__ or not callable(func): return False
    return all(p.annotation != empty for p in signature(func).parameters.values())

In [ ]:
def hi(who:str):
    "Say hi to `who`"
    return f"Hello {who}"

def hi2(who):
    "Say hi to `who`"
    return f"Hello {who}"

def hi3(who:str):
    return f"Hello {who}"

bye = "bye"

In [ ]:
assert is_usable_tool(hi)
assert not is_usable_tool(hi2)
assert not is_usable_tool(hi3)
assert not is_usable_tool(bye)

In [ ]:
#| export
def mk_toollist(syms):
    return "\n".join(f"- &`{sym.__name__}`: {sym.__doc__}" for sym in syms if is_usable_tool(sym))

In [ ]:
Markdown(mk_toollist([hi]))

- &`hi`: Say hi to `who`

In [ ]:
#| export
def import_gist(
    gist_id:str, # user/id or just id of gist to import as a module
    mod_name:str=None, # module name to create (taken from gist filename if not passed)
    add_global:bool=True, # add module to caller's globals?
    import_wildcard:bool=False, # import all exported symbols to caller's globals
    create_msg:bool=False # Add a message that lists usable tools
):
    "Import gist directly from string without saving to disk"
    fil = gist_file(gist_id)
    mod_name = mod_name or Path(fil['filename']).stem
    module = import_string(fil['content'], mod_name)
    glbs = currentframe().f_back.f_globals
    if add_global: glbs[mod_name] = module
    syms = getattr(module, '__all__', None)
    if syms is None: syms = [o for o in dir(module) if not o.startswith('_')]
    syms = [getattr(module, nm) for nm in syms]
    if import_wildcard:
        for sym in syms: glbs[sym.__name__] = sym
    if create_msg:
        pref = getattr(module, '__doc__', "Tools added to dialog:")
        add_msg(f"{pref}\n\n{mk_toollist(syms)}")
    return module

In [ ]:
import_gist(gistid)
importtest.testfoo

'testbar'

In [ ]:
import_gist.__doc__

'Import gist directly from string without saving to disk'

In [ ]:
import_gist(gistid, import_wildcard=True)
importtest.testfoo

In [ ]:
hi("Sarah")

In [ ]:
importtest.__all__

## Importing and Exporting Dialogs

We use json serialization of `list[dict]` to represent a dialogue where `dict` is a serialized `solveit.db_dc` `Message`. 

### Supported Fields

In [ ]:
from dialoghelper.db_dc import Message

In [ ]:
Message.__annotations__

{'sid': str | None,
 'mid': str | None,
 'content': str | None,
 'output': str | None,
 'input_tokens': int | None,
 'output_tokens': int | None,
 'msg_type': str | None,
 'time_run': str | None,
 'is_exported': int | None,
 'skipped': int | None,
 'did': int | None,
 'i_collapsed': int | None,
 'o_collapsed': int | None,
 'header_collapsed': int | None,
 'pinned': int | None}

When exporting and importing we intentionally leave out these fields:
- `sid` - to avoid duplicating sids across dialogues
- `mid` - to ensure `mid` respects ordering in the new dialogue
- `did` - since the dialogue will be imported into a different dialogue
- `time_run` - since a dialogue may be imported/exported to other environments where execution time is not comparable.

The following are required:
- `content`
- `msg_type`
- `output`

All other fields (`i_collapsed`, `header_collapsed`, ... etc) are technically optional. By default we read them on import if they are available. However, there may be other dialog-consuming applications (shell sage, discord buddy, others) we want to share dialogues with where these fields are not supported by the interface.

### Attachments

For now attachments are not included in the importing and exporting.

Attachments could be added as a future enhanement but would add some additional design considerations - if attachments refer to external artifacts, do the artifacts get copied? Do they get bundled as part of the export artifact? Do we create a new artifact entry with a new sid for the new dialogue or do we share artifacts?

### Implementation

In [ ]:
#| export

__EXPORT_FIELDS = set('content output input_tokens output_tokens msg_type is_exported skipped pinned i_collapsed o_collapsed header_collapsed'.split())

__REQUIRED_FIELDS = set('content output msg_type'.split())

In [ ]:
#| export
def export_dialog(filename: str, did:int=None):
    "Export dialog messages and optionally attachments to JSON"
    if did is None: did = find_dialog_id()
    db = get_db()
    msgs = db.t.message('did=? and (pinned=0 or pinned is null)', [did], order_by='mid')
    msg_data = [{k:getattr(msg,k) for k in __EXPORT_FIELDS if hasattr(msg, k)}
                for msg in msgs]
    result = {'messages': msg_data, 'dialog_name': db.t.dialog[did].name}
    with open(filename, 'w') as f: json.dump(result, f, indent=2)

In [ ]:
try:
    test_path = Path('test.json')
    export_dialog(test_path)
    test_msgs = test_path.read_json()
    print(test_msgs['messages'][1]['content'])
finally: test_path.unlink(missing_ok=True)

hello world


In [ ]:
#| export
def import_dialog(fname, add_header=True):
    "Import dialog messages from JSON file using `add_msg`"
    data = json.loads(Path(fname).read_text())
    for msg in data['messages'][::-1]:
        opts = {k:msg[k] for k in __EXPORT_FIELDS - __REQUIRED_FIELDS if k in msg}
        add_msg(msg.get('content',''), msg.get('msg_type','note'), msg.get('output',''), 'at_end', **opts)
    if add_header: add_msg(f"# Imported Dialog `{fname}`", 'note', placement='at_end')
    return f"Imported {len(data['messages'])} messages"

## export -

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()